In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
X, Y, Z = data_handling.preprocess_data(cell_images, cell_labels, cell_types)

print([cell_images.shape, len(cell_labels), len(cell_types)])
print([X.shape, len(Y), len(Z)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

@interact(n = (0, X.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(X[n], cmap = 'gray')
    axs.set_title(f'Class: {Y[n]} - Type: {Z[n]}')
    return None

[(2624, 300, 300), 2624, 2624]
[(2624, 300, 300, 3), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
X_train, Y_train, Z_train = X[: 2000], Y[: 2000], Z[: 2000]
X_val, Y_val, Z_val = X[2000 : ], Y[2000 : ], Z[2000 : ]

In [5]:
##
# Train classification model:
#
resnet152 = model_development.resnet152v2(input_shape = (300, 300, 3), 
    display_model_information = True)
history, training_time = model_development.train_classification_model(model = resnet152, 
    model_name = 'ResNet152v2', version = '20221001', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 10, batch_size = 4, validation_split_ratio = 0.2)

Model: "VGG19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 300, 300, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 300, 300, 3)       0         
                                                                 
 random_flip (RandomFlip)    (None, 300, 300, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 300, 300, 3)      0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 300, 300, 3)       0         
                                                                 
 random_translation (RandomT  (None, 300, 300, 3)      0         
 ranslation)                                                 